In [1]:
tk = None # 전역변수 선언

In [2]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# LSTM Training, Test
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# 불용어 사전 제작
stopwords = pd.read_csv("kostopword.txt") # 한글 불용어 파일 불러오기
stopwords = np.array(stopwords["stopword"].tolist()) # 불용어 사전을 비교할 수 있게 리스트 형태로 형변환
okt = Okt()

In [4]:
def data_preprocessing(data): # 입력값: 외부에서 가져온 데이터셋 / 출력값: Null 제거된 데이터셋
                              # 기능: 중복 제거, 한글과 공백을 제외하고 모두 제거, Null값 제거
    
    # 중복 여부 검사, nunique()는 중복값을 제외한 유니크한 값의 갯수를 카운팅 해줌.
    data['document'].nunique(), data['label'].nunique()
    
    # document 컬럼에서 중복인 내용이 있으면 중복 제거
    data.drop_duplicates(subset=['document'], inplace=True)
    # Null값이 존재하는 행 제거
    data = data.dropna(axis=0)
    
    # 한글과 공백을 제외하고 모두 제거하는 정규 표현식 사용
    data['document'] = data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
    
    # 다시한번 Null값이 존재하는지 확인
    data['document'] = data['document'].str.replace('^ +', "") # 공백만 있거나 빈 값을 가진 행이 있으면 빈문자열로 변경
    data['document'].replace('', np.nan, inplace=True) # 빈문자열을 Null값으로 변환
    
    # Train 데이터의 null 행 제거
    data = data.dropna(axis=0)
    
    # 최종적으로 한글을 제외한 데이터 및 모든 결측치를 제거한 dataset이 리턴됨.
    return data

In [5]:
def train_tokenizer(data):  # 입력값: Null값이 제거된 데이터셋 / 출력값: 불용어를 제거한 후 Null값도 제거한 데이터셋
                            # 기능: 불용어 제거, 단어 빈도수가 2회 이하인 단어 수를 찾아내고 공백 제거(공백 제거 목적)
    # 토큰화한 값을 X_data의 리스트로 넣어줌.
    X_data=[]
    # 데이터 처리가 완료된 데이터셋의 document 컬럼값들만 토큰화 진행
    for sentence in data['document']:
        # 형태소 분석기(Okt())에서 토큰화(한글은 띄어쓰기) 실행. stem=True로 일정 수준 정규화(동사,명사화)
        X_tmp = okt.morphs(sentence, stem=True)
        X_tmp = [word for word in X_tmp if not word in stopwords] # 불용어 사전에 없으면 리스트에 추가
        X_data.append(X_tmp)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_data)
    
    # 등장 빈도수가 3회 미만인 단어들이 이 데이터에서 얼만큼 비중을 차지하는지 확인
    bindo = 3 # 단어의 등장 빈도수 기준
    rare_cnt = 0 # 등장 빈도수가 bindo보다 작은 단어의 개수를 카운트
    total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
    rare_freq = 0 # 등장 빈도수가 bindo보다 작은 단어의 빈도수 총 합
    total_cnt = len(tokenizer.word_index) # 단어의 수
    
    # key-value 형태로 저장
    for key, value in tokenizer.word_counts.items():
        total_freq = total_freq + value

        # 단어의 등장 빈도수가 bindo보다 작으면 아래 if 문을 실행
        if(value < bindo):
            rare_cnt = rare_cnt + 1
            rare_freq = rare_freq + value

    # 단어 빈도수가 2회 이하인 단어들은 제외
    # 0번을 고려해서 크기는 +1을 해준다.
    voca_size = total_cnt - rare_cnt + 1
    
    tokenizer = Tokenizer(voca_size)
    tokenizer.fit_on_texts(X_data)
    
    y_data = np.array(data['label'])
    global tk
    tk = tokenizer
    X_data, y_data = train_padding(X_data, y_data)
    
    return X_data, y_data, voca_size

In [6]:
def train_padding(X_data, y_data):
    
    X_data = tk.texts_to_sequences(X_data) # texts_to_sequences > 단어들에 순번을 지정

    # empty samples 제거
    drop_data = [index for index, sentence in enumerate(X_data) if len(sentence) < 1] # 단어가 1개 미만, 즉 비어있는 데이터 제거

    # 빈 샘플 제거
    X_data = np.delete(X_data, drop_data, axis=0) # X_data에서 drop_data을 사용해서 제거
    y_data = np.delete(y_data, drop_data, axis=0)
    
    # 전체 훈련 데이터중 대부분의 데이터 길이가 35 이하이므로 모든 샘플의 길이를 35으로 조정
    X_data = pad_sequences(X_data, maxlen = 35)
    
    return X_data, y_data

In [7]:
def test_tokenizer(data):  # 입력값: Null값이 제거된 데이터셋 / 출력값: 불용어를 제거한 후 Null값도 제거한 데이터셋
                            # 기능: 불용어 제거, 단어 빈도수가 2회 이하인 단어 수를 찾아내고 공백 제거(공백 제거 목적)
    X_data=[]
    for sentence in data['document']:
        # 형태소 분석기(Okt())에서 토큰화(한글은 띄어쓰기) 실행. stem=True로 일정 수준 정규화(동사,명사화)
        X_tmp = okt.morphs(sentence, stem=True)
        X_tmp = [word for word in X_tmp if not word in stopwords] # 불용어 사전에 없으면 리스트에 추가
        X_data.append(X_tmp)

    y_data = np.array(data['label'])

    X_data = test_padding(X_data)
    
    return X_data, y_data

In [8]:
def test_padding(X_data):

    X_data = tk.texts_to_sequences(X_data) # texts_to_sequences > 단어들에 순번을 지정

    # 전체 훈련 데이터중 대부분의 데이터 길이가 35 이하이므로 모든 샘플의 길이를 35으로 조정
    X_data = pad_sequences(X_data, maxlen = 35)

    return X_data

In [9]:
def TrainAndTest(X_data, y_data, X_data2, y_data2, voca_size): # 입력값: / 출력값: /함수의 기능:

    model = Sequential()
    # 케라스 시퀀셜을 하나의 입력과 출력을 가짐 다음 셀로 이동시키는 기능!
    model.add(Embedding(voca_size, 100)) # 정수화 시키는 것(전체 단어의 집합을 벡터의 크기를 100으로 임베딩한다.)
    model.add(LSTM(128)) # 128개의 셀로 LSTM을 사용
    model.add(Dense(1, activation='sigmoid')) # 처음 인자 출력 갯수, 활성 함수는 sigmoid사용(binary classification)

    # 검증 데이터 손실이 증가하면, 과적합 위험. 검증 데이터 손실이 4회 증가하면 학습을 조기 종료
    # ModelCheckpoint를 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델 저장
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    # 조건에 해당하면 처리 과정을 멈춰라, val_loss를 모니터링 하고 손실값이기 때문에 mode는 최소인 min으로 설정
    # verbose=1은 언제 멈췄는지 화면에 찍어주는 역할, patience 성능이 증가하지 않을때 몇번 더 시도할 것인지 지정
    mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
    # 정확도를 모니터하여 최대값을 가지는 파라미터 모델을 best_model.h5 이름으로 저장한다.

    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    # 모델을 컴파일 할때는 정규화기, 손실함수, 평가지표 3가지를 설정한다.
    # rmsprop는 최신 기울기의 반영 비율이 더 높은 방식, binary_corssentropy는 output layer가 sigmod일떄 사용
    # 분류값이기 때문에 평가지표는 정확도를 사용한다.
    history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=60, validation_split=0.2)
    # epcohs는 15회를 사용하고 20%의 데이터를 검증용으로 분류한다.
    # 얼리스탑과 모델체크포인트를 콜백하여 사용한다.

    loaded_model = load_model('best_model.h5')
    # 모델체크포인트로 성능이 가장 좋은 모델을 가져온다.
    # 성능이 가장 좋은 모델로 테스트용 데이터를 검증한다.
    print("\n 테스트 정확도 : %0.4f" % (loaded_model.evaluate(X_data2, y_data2)[1]))
    # 0번 인덱스는 loss, 1번 인덱스는 정확도

In [10]:
def sentimemt_predict(new_sentence):
#     a, b, c, tokenizer = data_prediction(np.array(new_sentence))
    loaded_model = load_model('best_model.h5')
    new_sentence = okt.morphs(new_sentence, stem=True) #토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] #불용어 
    
    pad_new = test_padding([new_sentence])
#     encoded = tk.texts_to_sequences([new_sentence]) #정수 인코딩
#     pad_new = pad_sequences(encoded, maxlen = 35) #패딩
    score = float(loaded_model.predict(pad_new)) #예측
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [11]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [12]:
train_data = data_preprocessing(train_data)
test_data = data_preprocessing(test_data)

In [13]:
X_train, y_train, voca_size = train_tokenizer(train_data)
X_test, y_test = test_tokenizer(test_data)

In [14]:
# Training(X_train, y_train, voca_size)
TrainAndTest(X_train, y_train, X_test, y_test, voca_size)

Epoch 1/15
1933/1934 [============================>.] - ETA: 0s - loss: 0.3922 - acc: 0.8214
Epoch 00001: val_acc improved from -inf to 0.83642, saving model to best_model.h5
1934/1934 [==============================] - 72s 37ms/step - loss: 0.3922 - acc: 0.8214 - val_loss: 0.3654 - val_acc: 0.8364
Epoch 2/15
1933/1934 [============================>.] - ETA: 0s - loss: 0.3293 - acc: 0.8559
Epoch 00002: val_acc improved from 0.83642 to 0.85183, saving model to best_model.h5
1934/1934 [==============================] - 72s 37ms/step - loss: 0.3293 - acc: 0.8559 - val_loss: 0.3374 - val_acc: 0.8518
Epoch 3/15
1933/1934 [============================>.] - ETA: 0s - loss: 0.3036 - acc: 0.8705
Epoch 00003: val_acc improved from 0.85183 to 0.85690, saving model to best_model.h5
1934/1934 [==============================] - 73s 38ms/step - loss: 0.3036 - acc: 0.8705 - val_loss: 0.3303 - val_acc: 0.8569
Epoch 4/15
1934/1934 [==============================] - ETA: 0s - loss: 0.2851 - acc: 0.8810
E

In [15]:
sentimemt_predict('재밌어요')

94.77% 확률로 긍정 리뷰입니다.



In [16]:
sentimemt_predict('한번쯤은 볼만한 영화')

63.19% 확률로 긍정 리뷰입니다.



In [17]:
sentimemt_predict('쓰레기')

98.68% 확률로 부정 리뷰입니다.



In [18]:
sentimemt_predict('보라고 하고싶진 않지만 한번은 보세요')

65.84% 확률로 부정 리뷰입니다.



In [19]:
sentimemt_predict('꼭 보고싶다면 말리진 않을게요')

82.11% 확률로 긍정 리뷰입니다.



In [20]:
sentimemt_predict('이걸 돈주고 본다고?')

96.93% 확률로 부정 리뷰입니다.



In [22]:
sentimemt_predict('개행개행')

61.64% 확률로 부정 리뷰입니다.

